# example code for checking installation and functionality of the code

In [1]:
import pyomo.environ as pyo
from pyomo2h5 import PyomoHDF5Saver, ConstraintTracker

## Create pyomo model

In [2]:
# Create Model
m = pyo.ConcreteModel("name")

m.Scenarios = pyo.RangeSet(3)

m.I = pyo.RangeSet(5)

m.kk = pyo.Set(initialize=[1,43,"a"])

m.p = pyo.Param(initialize=5,doc="this is a parameter")

m.x = pyo.Var(m.I, within=pyo.NonNegativeReals,doc="this is an indexed variable")

@m.Objective(doc="This is an Objective", sense=pyo.minimize)
def obj(m):
    return sum(m.x[i] for i in m.I)

@m.Constraint(doc="This is a Constraint")
def cons1(m):
    return m.x[3]<=4

@m.Block()
def scenario(block):
    block.Scenarios = pyo.RangeSet(5)
    block.x = pyo.Var(block.Scenarios, within=pyo.NonNegativeIntegers)
    @block.Constraint()
    def block_con1(block):
        return sum(block.x[j] for j in block.Scenarios) >= 34

@m.Constraint(m.I, doc="This is a Constraint")
def con1(m, i):
    return m.x[i] >= m.scenario.x[i]/10

## Solve Model

In [3]:
filename = "test4"

solver = pyo.SolverFactory("gurobi", solver_io="python")
solver.options["LogFile"] = filename + ".log"
solver.options["TimeLimit"] = 60
results = solver.solve(m, tee=True)

Set parameter OutputFlag to value 1
Set parameter LogFile to value "test4.log"
Set parameter TimeLimit to value 60
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: AMD Ryzen 7 PRO 5850U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  60

Optimize a model with 7 rows, 10 columns and 16 nonzeros
Model fingerprint: 0xed9b7ad6
Variable types: 5 continuous, 5 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 3e+01]
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 3.4 



Optimal solution found (tolerance 1.00e-04)
Best objective 3.400000000000e+00, best bound 3.400000000000e+00, gap 0.0000%


## Save to h5

In [5]:
with PyomoHDF5Saver(filename, force=False) as saver:
    saver.save_instance(m, results, solver_options=solver.options)

ValueError: Unable to synchronously create dataset (name already exists)

## Add metadata

In [5]:
metadata = {
    "Metadata": {"Written by": "Julius"},
    "Notes": {
        "Content": "This is an example save.",
        "Metadata": {"Reviewed": "Yes"}
    }
}

with PyomoHDF5Saver(filename, force=False, mode="a") as saver:
    saver._save_annotated_dict(metadata)

## Load instance from filename

In [6]:
with PyomoHDF5Saver(filename,mode="r") as saver:
    instance2 = saver.load_instance()

instance2.x.pprint()

x : this is an indexed variable
    Size=5, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  -0.0 :  None : False : False : NonNegativeReals
      2 :     0 :  -0.0 :  None : False : False : NonNegativeReals
      3 :     0 :   3.4 :  None : False : False : NonNegativeReals
      4 :     0 :  -0.0 :  None : False : False : NonNegativeReals
      5 :     0 :  -0.0 :  None : False : False : NonNegativeReals


## Add constraints that are tracked
handy for computing pareto fronts to keep track of added constraints

In [7]:
tracker = ConstraintTracker()


@m.Constraint()
def x1_min2(m):
    return m.x[3]>=1

# Add some constraints
tracker.add(m.x1_min2)
# tracker.add(m, m.x[2] <= 5, name="x2_max2")

# tracker.delete(m, "x1_min33")

# Show them
tracker.print_constraints()

with PyomoHDF5Saver(filename) as saver:
    saver.save_tracked_constraints(tracker, "Additional_constraints")

x1_min2: x1_min2
